In [2]:
# Autoreload extension
%load_ext autoreload
%autoreload 2

In [3]:
# Load geostacks
from geostacks import SpatialIndexLS8

# Load UI components
import ipyleaflet as ilfl
from ipyleaflet import Polygon
import ipywidgets as iwg

In [4]:
ls8 = SpatialIndexLS8()
ls8.data

,path,row,lat_CTR,lon_CTR,lat_UL,lon_UL,lat_UR,lon_UR,lat_LL,lon_LL,lat_LR,lon_LR
0,1,2,80.002493,-4.197763,81.205697,-2.730017,79.717460,2.594563,80.144957,-11.291900,78.789287,-5.405237
1,1,3,79.111023,-10.561457,80.332344,-9.994770,78.957946,-4.156684,79.130143,-17.075937,77.882976,-11.061501
2,1,4,78.118527,-15.970556,79.344246,-16.045440,78.079664,-10.018548,78.034189,-21.909139,76.886141,-15.950291
3,1,5,77.048224,-20.471403,78.269010,-20.978442,77.105158,-14.988030,76.879455,-25.870470,75.819367,-20.086304
4,1,6,75.902095,-24.338152,77.113394,-25.133782,76.041404,-19.307509,75.661913,-29.246411,74.680825,-23.697966
...,...,...,...,...,...,...,...,...,...,...,...,...
21898,233,242,80.008794,44.207091,80.154264,51.282248,78.799314,45.389529,81.215211,42.713538,79.726284,37.379350
21899,233,243,80.760793,36.728885,81.052819,44.273388,79.584925,38.823265,81.923478,34.042857,80.327864,29.693888
21900,233,244,81.338812,28.123821,81.798007,35.890003,80.221703,31.325044,82.424145,24.005498,80.744047,21.177876
21901,233,245,81.705630,18.551148,82.326232,26.541583,80.663323,23.323882,82.678136,12.476933,80.951763,11.702585


In [5]:
# ls8_index = SpatialIndexLS8('../../LANDSAT_8_C1_cornerpoints.csv')
# ls8_index.read()
# ls8_index.corner_pts_df

In [8]:
# query_pt = [-50., 69.]
idxs = ls8.query_pathrow(69, -50)   # lat, lon
print(len(idxs.data.index))
ls8.data.loc[idxs.data.index]

10


,path,row,lat_CTR,lon_CTR,lat_UL,lon_UL,lat_UR,lon_UR,lat_LL,lon_LL,lat_LR,lon_LR
695,8,11,69.606470,-47.813174,70.756261,-49.045943,69.997303,-44.451408,69.154025,-51.050718,68.439813,-46.725544
696,8,12,68.279699,-49.506130,69.419648,-50.743468,68.700888,-46.367352,67.801954,-52.529493,67.122236,-48.403704
782,9,11,69.606482,-49.355231,70.757156,-50.589565,69.997520,-45.990513,69.153716,-52.595528,68.438861,-48.266305
783,9,12,68.279697,-51.046291,69.420484,-52.285488,68.700913,-47.904523,67.801817,-54.072478,67.121321,-49.942257
866,10,11,69.606439,-50.895321,70.756467,-52.132700,69.996319,-47.530333,69.154836,-54.136166,68.439445,-49.803544
6845,81,233,69.606467,-48.202833,69.158069,-44.965508,68.442808,-49.295142,70.763699,-46.977141,70.002875,-51.580448
6915,82,233,69.606440,-49.749759,69.160805,-46.513479,68.444886,-50.847126,70.761497,-48.518743,70.000130,-53.125468
7017,83,232,68.279726,-49.604053,67.806027,-46.581815,67.125546,-50.709373,69.426907,-48.374276,68.706648,-52.756036
7018,83,233,69.606447,-51.293647,69.158697,-48.052981,68.442311,-52.389424,70.764101,-50.064050,70.002091,-54.674559
7087,84,232,68.279691,-51.145138,67.808695,-48.124313,67.127863,-52.255251,69.424457,-49.910379,68.703970,-54.294948


In [5]:
class geostacks_ui():
    
    def __init__(self, zoom=4, lon=-50.,lat=69., spatial_index=None):
        
        self.zoom = zoom
        self.lat = lat
        self.lon = lon
        self.mainmap = None
        self.marker = None
        self.idxs = None
        self.ui_title = None
        self.prlist = []
        self.menuleft = None     # path / row menu
        self.pr_selection = None
        self.map_polygon = None
        self.spatial_index = spatial_index
        self.record = None       # temporary attribute to store select spatial record
        self.output = None       # print message output (unused for now)
        self.results = None      # store results (unused for now)
        
    def init_panelleft(self):
        self.ui_title = iwg.HTML("<h2>Drag the marker to your region of interest</h2>")
        self.idxs = self.spatial_index.query_pathrow(self.lat,self.lon)
        self.prlist = [('{:03d}/{:03d}'.format(self.spatial_index.corner_pts_df.loc[i, 'path'], 
                                              self.spatial_index.corner_pts_df.loc[i, 'row' ]), i) for i in self.idxs]
        self.menuleft = iwg.Select(options=self.prlist, description='Path/Row:', rows=15)
        
    def init_map(self):
        self.mainmap = ilfl.Map(basemap=ilfl.basemaps.Gaode.Satellite,
                                center=[self.lat, self.lon], zoom=self.zoom)
        self.marker = ilfl.Marker(location=[self.lat, self.lon], draggable=True)
        self.mainmap.add_layer(self.marker)
        self.pr_selection = self.idxs[0]
        self.record = self.spatial_index.corner_pts_df.loc[self.pr_selection]
        self.map_polygon = Polygon(
            locations=[(self.record.lat_UL, self.record.lon_UL), (self.record.lat_UR, self.record.lon_UR), (self.record.lat_LR, self.record.lon_LR), (self.record.lat_LL, self.record.lon_LL)],
            color="blue")
        self.mainmap.add_layer(self.map_polygon)
        
    def gen_ui(self, spatial_index=None):
        if self.spatial_index is None:
            self.spatial_index = spatial_index
        
        self.init_panelleft()
        self.init_map()
        
        self.marker.observe(self._on_location_changed, 'location')
        self.menuleft.observe(self._on_menuleft_selection_changed, names='value')
        leftside = iwg.VBox([self.ui_title, self.menuleft])
        leftside.layout.align_items = 'center'
        return iwg.AppLayout(left_sidebar=leftside, center=self.mainmap)

    # ==== leftmenu update when map marker loc changes
    
    def _on_location_changed(self, event):
        # self.query_pt = [self.marker.location[-1], self.marker.location[0]]
        self.lat = self.marker.location[0]
        self.lon = self.marker.location[-1]
        self.idxs = self.spatial_index.query_pathrow(self.lat, self.lon)
        self.prlist = [('{:03d}/{:03d}'.format(self.spatial_index.corner_pts_df.loc[i, 'path'], 
                                               self.spatial_index.corner_pts_df.loc[i, 'row']), i) for i in self.idxs]
        self.menuleft.options = self.prlist
        
    # ==== map polygon update when leftmenu selection changes

    def _on_menuleft_selection_changed(self, change):
        self.pr_selection = change['new']
        self.record = self.spatial_index.corner_pts_df.loc[self.pr_selection]
        self.map_polygon.locations = [(self.record.lat_UL, self.record.lon_UL), (self.record.lat_UR, self.record.lon_UR), (self.record.lat_LR, self.record.lon_LR), (self.record.lat_LL, self.record.lon_LL)]

In [7]:
cpanel = geostacks_ui(spatial_index=ls8_index)
cpanel.gen_ui()

AppLayout(children=(VBox(children=(HTML(value='<h2>Drag the marker to your region of interest</h2>'), Select(d…